In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import cv2

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import cv2
import torch
import timm
import time
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2


c:\Users\Anas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

BATCH_SIZE = 32
IMG_SIZE = (128, 128)
DATASET_PATH = "../../Datasets/LFR Dataset/split"

# Load datasets
train_ds = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

test_ds = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "test"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# # Enable prefetching for performance
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
# test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
# # Check how many samples are in each dataset



Found 98505 files belonging to 3 classes.
Found 21108 files belonging to 3 classes.
Found 21111 files belonging to 3 classes.


In [3]:
from PIL import Image
import os

def remove_corrupt_images(folder_path):
    removed = 0
    for root, _, files in os.walk(folder_path):
        for f in files:
            file_path = os.path.join(root, f)
            try:
                img = Image.open(file_path)
                img.verify()  # Just verifies, doesn't load entire image
            except Exception as e:
                print(f"Removing corrupt image: {file_path}")
                os.remove(file_path)
                removed += 1
    print(f"Removed {removed} corrupt images")

# Run on your dataset
remove_corrupt_images("../../Datasets/LFR Dataset/split")


Removed 0 corrupt images


In [4]:
print(f"Train dataset size: {len(train_ds)}")
print(f"Test dataset size: {len(test_ds)}")
print(f"val dataset size: {len(val_ds)}")

Train dataset size: 3079
Test dataset size: 660
val dataset size: 660


In [5]:
from tensorflow.keras import layers
import tensorflow as tf  # Also make sure this is imported

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomBrightness(0.1),
    layers.GaussianNoise(0.02),
])


In [6]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input

base_model = EfficientNetB0(input_shape=IMG_SIZE + (3,),
                            include_top=False,
                            weights='imagenet')
base_model.trainable = False  # Freeze base

inputs = Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(3, activation='softmax')(x)  # 3 classes

model = Model(inputs, outputs)


In [7]:
model.compile(
    optimizer='adam',  # Optimizer of your choice
    loss='sparse_categorical_crossentropy',  # Since you're using integer labels
    metrics=['accuracy']
)


In [8]:
history = model.fit(
    train_ds,
    epochs=10,  # Start with just one epoch for testing
    validation_data=val_ds,
    verbose=1  # Make sure you're able to see logs for debugging
)


Epoch 1/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 601s 193ms/step - accuracy: 0.6986 - loss: 0.6796 - val_accuracy: 0.7330 - val_loss: 0.6019
Epoch 2/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 590s 192ms/step - accuracy: 0.7189 - loss: 0.6254 - val_accuracy: 0.7296 - val_loss: 0.5957
Epoch 3/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 567s 184ms/step - accuracy: 0.7212 - loss: 0.6226 - val_accuracy: 0.7291 - val_loss: 0.6013
Epoch 4/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 567s 184ms/step - accuracy: 0.7215 - loss: 0.6231 - val_accuracy: 0.7315 - val_loss: 0.5954
Epoch 5/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 580s 188ms/step - accuracy: 0.7218 - loss: 0.6219 - val_accuracy: 0.7339 - val_loss: 0.5977
Epoch 6/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 593s 193ms/step - accuracy: 0.7197 - loss: 0.6254 - val_accuracy: 0.7335 - val_loss: 0.5894
Epoch 7/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 589s 191ms/step - accuracy: 0.7200 - loss: 0.6228 - val_accuracy: 0.7318 - val_loss: 0.6013
Epoch 8/10
3079/3079 ━━━━━━━━━━━━━━━━━━━━ 592s 192ms/step - ac

In [21]:
model.save("../../models/face_dir_further.keras")


In [ ]:
from keras.models import load_model
model = load_model("face_direction_model.keras")
